<a href="https://colab.research.google.com/github/bdandersen-berkeley/mids/blob/master/data_management.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MIDS W207 Group 9 - Final Project - Data Management

## Colaboratory Notebook Preconditions

**This Colab notebook requires access to a Google Drive.**  Project data is retrieved from a folder on the Google Drive associated with the Google account with which this notebook is executing.

This data folder may exist anywhere beneath the Google Drive.  Unfortunately, mounted Google Drives do not have access to folders and files shared by others.

If the project data files do not reside on the Google Drive of the current Google account, please contact any of the following Group 9 members for assistance:

* Brad Andersen - bdandersen@berkeley.edu
* Stephanie Mather - stephanie.mather@berkeley.edu
* Sonal Thakkar - sonalthakkar@berkeley.edu

In [0]:
# Necessary only if Turi Create is leveraged within the notebook

# import sys
# print("Python runtime:", sys.version_info)
# assert 3 == sys.version_info.major and 6 == sys.version_info.minor, "Python runtime must be version 3.6.x"

### Identifying the Pathname for Project Resources

This Colab notebook will mount the current user's Google Drive beneath `/content/drive`.   Please specify the pathname of the project resource folder as it is maintained on the current user's Google Drive.

*This is the folder beneath which subfolders **data** and **py** are located.*

In [0]:
gdrive_project_pathname = '/My Drive/MIDS/W207/Final Project'  #@param {type: "string"}

In [6]:
import os.path
from google.colab import drive

# Mount the current user's Google Drive
GOOGLE_DRIVE_MOUNT_POINT = "/content/drive"
print("Mounting Google Drive beneath %s" % GOOGLE_DRIVE_MOUNT_POINT)
drive.mount(GOOGLE_DRIVE_MOUNT_POINT, force_remount = True)

# Build the pathname to the project's folder residing beneath the current user's Google Drive
if not gdrive_project_pathname.startswith("/"):
  gdrive_project_pathname = "/" + gdrive_project_pathname
abs_project_pathname = GOOGLE_DRIVE_MOUNT_POINT + gdrive_project_pathname
print("Project folder: %s" % gdrive_project_pathname)

# Check that the subdirectories anticipated beneath the Google Drive project folder exist
# by checking for the presence of the DO_NOT_DELETE.txt file
for subfolder in ["data", "py"]:
    if not os.path.exists(abs_project_pathname + "/" + subfolder + "/DO_NOT_DELETE.txt"):
        raise FileNotFoundError("Required subfolder '" + subfolder + "' does not exist beneath the Google Drive project folder")
print("Project subfolders successfully verified")

abs_data_pathname = abs_project_pathname + "/data"
abs_py_pathname = abs_project_pathname + "/py"

Mounting Google Drive beneath /content/drive
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
Project folder: /My Drive/MIDS/W207/Final Project
Project subfolders successfully verified


### Dependencies

In [8]:
import sys

sys.path.append(abs_py_pathname)
import data
import eda
import ml_utl
import IPython

! pip install -U mne

Requirement already up-to-date: mne in /usr/local/lib/python3.6/dist-packages (0.18.2)


## EDA of Seizure Dataset 

### Accessing Data

Data associated with the ***UPenn and Mayo Clinic's Seizure Detection Challenge*** Kaggle competition were distributed in a single gzipped tar file approximately 10.8 Gb in size:

```
(base) brad@ryzen:~/work/tmp$ ls -la clips.tar.gz
-rw-r--r-- 1 brad brad 11621036129 Jul  1 17:21 clips.tar.gz
```

The data file was comprised of 58,837 individual MATLAB files, each representing one second's worth of EEG (electroencephalogram) data gathered from a patient, referred to as a "clip".  Uncompressed and extracted, these files consumed approximately 49 Gb of disk space:

```
(base) brad@ryzen:~/work/tmp$ du -bc Volumes | grep total
49093339235	total
```

Some data within the data file were useful to our analysis (e.g. human patient ictal and interictal training data) while others we disregarded (e.g. all data gathered from canine subjects).  Compression and the serialized nature of tar files made accessing data in realtime impractical.  The size of all individual files proved difficult to maintain on a shared drive, and exceeded the memory capacity of our Jupyter notebooks, both hosted on Google's Colaboratory site or local).  A more efficient means of managing the data was required.






#### Initial Separation of Related Files

We chose to focus our analysis upon human subjects (i.e. "patients") and disregard data gathered from canines.  This, however, only reduced the amount of data required in our analysis by approximately 1.1 Gb.  Because data were gathered from individual patients -- a key feature in our analysis -- and previously separated into MATLAB files representing training and test data, we chose to separate each patient's data into two gzipped tar files: one containing test data, the other labeled training data.  These files were then copied to a shared folder accessible to our analysis.

#### MATLAB Format

Data in each clip file were maintained and the resulting file formatted using MATLAB.  SciPy.org's `loadmat` function was used to load these files into memory.

## Competition Data

The description of the data provided on the Kaggle competition's site is somewhat confusing.  Paraphrased and separated into bullet points, data are captured and organized as follows:

* The training data is organized into 1-second EEG clips labeled "Ictal" for seizure data segments, or "Interictal" for non-seizure data segments
* Training data are arranged sequentially; testing data are in random order
* Ictal training and testing data segments are provided covering the entire seizure
* Interictal data segments are provided covering approximately the mean seizure duration for each subject
* Starting points for the interictal data segments were chosen randomly from the full data record, with the restriction that no interictal segment be less than one hour before or after a seizure

Naming conventions for the MATLAB files identified the subject type (e.g. human or canine subject), subject number, test or training data, and the order of the clip in the sequence of EEG measurements.  For example,

```
Patient_5_interictal_segment_2581.mat
Dog_2_test_segment_1465.mat
```

Within each MATLAB file is a dictionary containing the following.  Upon analysis, we found that the structure and naming of data are slightly different than what is documented within the Kaggle competition.

|  dict Header  |  Data Description  |
|  ---  |  ---  |
|  **data**  |  Matrix of EEG sample values arranged row x column as electrode x time  |
|  **latency**  |  time in seconds between the expert-marked seizure onset and the first data point in the data segment (in ictal training segments only)  |
|  **freq**  |  Sampling frequency - Number of data samples representing 1 second of EEG data (non-integer values represent an average across the full data record and may reflect missing EEG samples)  |
|  **channels**  |  List of electrode names corresponding to the rows in the data field  |

A summary of the data is shown below (void of EEG and electrode data):

In [22]:
CLIP_FILENAME = "Patient_1_ictal_segment_1.mat"
clip = eda.clip_load(abs_data_pathname, CLIP_FILENAME)
clip.keys()

dict_keys(['__header__', '__version__', '__globals__', 'data', 'freq', 'channels', 'latency'])

In [23]:
eda.clip_summary(abs_data_pathname, CLIP_FILENAME)

Filename:           Patient_1_ictal_segment_1.mat
Subject:            Patient_1
Data class:         ictal
Segment:            1
Data:
  Shape:            (68, 500)
  Min (volts):      -680.8720
  Max (volts):      890.5680
  Duration (sec):   unknown
Latency (sec):      0.0000
Samples:            unknown
Electrodes:         68


#### Electrode (i.e. "channel") Identification and Measurement Data

Accessing channel data was somewhat involved, as data were maintained in a hierarchy of arrays for which the hierachy seemed to serve no purpose.  Unfortunately, each MATLAB file only maintained a string token identifying the electrode associated with a given row of EEG measurements.

*It is possible that having only a string token for each electrode may have led to some loss of positional data when integrating the clip file with traditional EEG visualisation tools.*

In [25]:
print("String tokens identifying electrodes:")
for electrode in range(10):
    print(clip["channels"][0][0][electrode])
print("...")
print("Total number of electrodes providing measurements in clip: %d" % len(clip["channels"][0][0]))

String tokens identifying electrodes:
['LFG1']
['LFG10']
['LFG11']
['LFG12']
['LFG13']
['LFG14']
['LFG15']
['LFG16']
['LFG17']
['LFG18']
...
Total number of electrodes providing measurements in clip: 68


Electrode measurements are represented as voltages, and arranged in a matrix as follows:

In [26]:
clip_df = eda.create_clip_df(abs_data_pathname, CLIP_FILENAME)
clip_df[0].head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499
LFG1,204.984,213.984,217.984,197.984,186.984,177.984,174.984,173.984,156.984,141.984,143.984,128.984,119.984,102.984,99.984,92.984,80.984,97.984,112.984,143.984,161.984,183.984,178.984,175.984,187.984,180.984,162.984,152.984,137.984,144.984,145.984,149.984,165.984,178.984,192.984,190.984,158.984,112.984,54.984,46.984,...,-280.016,-292.016,-303.016,-295.016,-264.016,-254.016,-246.016,-228.016,-221.016,-212.016,-226.016,-248.016,-240.016,-262.016,-239.016,-205.016,-190.016,-177.016,-167.016,-169.016,-156.016,-154.016,-152.016,-113.016,-79.016,-58.016,-53.016,-75.016,-88.016,-85.016,-119.016,-112.016,-105.016,-120.016,-115.016,-142.016,-179.016,-195.016,-216.016,-214.016
LFG10,-138.314,-127.314,-111.314,-148.314,-179.314,-201.314,-210.314,-198.314,-210.314,-211.314,-230.314,-260.314,-285.314,-290.314,-300.314,-272.314,-223.314,-173.314,-135.314,-117.314,-127.314,-131.314,-139.314,-156.314,-175.314,-207.314,-236.314,-237.314,-237.314,-238.314,-236.314,-199.314,-127.314,-72.314,-35.314,-23.314,-8.314,-1.314,-5.314,9.686,...,-250.314,-259.314,-273.314,-275.314,-249.314,-236.314,-235.314,-240.314,-223.314,-204.314,-163.314,-141.314,-118.314,-125.314,-127.314,-143.314,-153.314,-152.314,-159.314,-154.314,-161.314,-166.314,-159.314,-156.314,-161.314,-158.314,-158.314,-145.314,-118.314,-81.314,-37.314,14.686,68.686,93.686,117.686,105.686,63.686,17.686,-17.314,-40.314
LFG11,-64.030,-57.030,-28.030,-50.030,-117.030,-162.030,-196.030,-189.030,-216.030,-222.030,-229.030,-240.030,-243.030,-255.030,-247.030,-233.030,-226.030,-214.030,-222.030,-213.030,-233.030,-231.030,-232.030,-220.030,-215.030,-220.030,-231.030,-266.030,-301.030,-315.030,-307.030,-270.030,-240.030,-225.030,-189.030,-174.030,-165.030,-169.030,-191.030,-195.030,...,-119.030,-129.030,-158.030,-182.030,-167.030,-128.030,-62.030,-40.030,-45.030,-47.030,-69.030,-85.030,-59.030,-74.030,-72.030,-91.030,-93.030,-79.030,-99.030,-101.030,-88.030,-61.030,-14.030,-13.030,-37.030,-41.030,-62.030,-70.030,-65.030,-46.030,-60.030,-47.030,-34.030,-52.030,-39.030,-40.030,-46.030,-55.030,-83.030,-66.030
LFG12,-167.764,-152.764,-158.764,-187.764,-195.764,-220.764,-259.764,-299.764,-332.764,-373.764,-420.764,-435.764,-436.764,-442.764,-423.764,-388.764,-368.764,-350.764,-339.764,-368.764,-415.764,-424.764,-440.764,-460.764,-451.764,-415.764,-363.764,-325.764,-349.764,-365.764,-362.764,-349.764,-324.764,-271.764,-193.764,-176.764,-197.764,-216.764,-247.764,-245.764,...,-160.764,-213.764,-284.764,-320.764,-265.764,-218.764,-185.764,-152.764,-170.764,-170.764,-127.764,-111.764,-78.764,-29.764,45.236,127.236,159.236,187.236,191.236,261.236,270.236,240.236,237.236,193.236,156.236,141.236,147.236,152.236,157.236,182.236,180.236,175.236,171.236,154.236,174.236,172.236,196.236,234.236,257.236,293.236
LFG13,-139.808,-118.808,-87.808,-102.808,-94.808,-116.808,-126.808,-147.808,-177.808,-196.808,-201.808,-219.808,-267.808,-316.808,-330.808,-298.808,-231.808,-151.808,-85.808,-45.808,-54.808,-78.808,-118.808,-142.808,-118.808,-72.808,-40.808,-20.808,-6.808,-6.808,-25.808,-37.808,-72.808,-67.808,-43.808,-48.808,-46.808,-49.808,-65.808,-42.808,...,-89.808,-114.808,-155.808,-170.808,-151.808,-141.808,-118.808,-106.808,-97.808,-74.808,-54.808,-42.808,-12.808,3.192,57.192,77.192,100.192,140.192,171.192,207.192,213.192,195.192,190.192,176.192,151.192,154.192,152.192,143.192,143.192,139.192,147.192,176.192,205.192,204.192,220.192,217.192,211.192,228.192,248.192,286.192
